In [ ]:
import requests
import json
import base64

base_url = 'http://localhost:8081/v1/pytorch_api/'
detect_endpoint = 'detect'

async_url = base_url + detect_endpoint
# https://stackoverflow.com/questions/3715493/encoding-an-image-file-with-base64
def read_tif_as_base64_str(tif_path):
    ENCODING = 'utf-8'
    with open(tif_path, "rb") as image_file:
        gtiff_bytes = image_file.read()
        gtiff_bytes_b64_encoded = base64.b64encode(gtiff_bytes)
        return gtiff_bytes_b64_encoded.decode(ENCODING)
tif_path = "../images/aoi_restricted_LT05_CU_013008_20050828_20190107_C01_V0_-485025_2041605.tif"
payload = {'data': read_tif_as_base64_str(tif_path),
           'outname': "results_LT05_CU_013008_20050828_20190107_C01_V0_-485025_2041605.tif",
           "score_threshold": .8
            }
# r = requests.post(async_url, data=read_tif_as_base64(tif_path))
r = requests.post(async_url, json=payload)
print(r.text)

In [ ]:
taskid = int(r.text.split(": ")[1])
url = f"http://localhost:8081/v1/pytorch_api/task/{taskid}"

payload={}
headers = {}

response = requests.request("GET", url, headers=headers, data=payload)

print(response.text)


### Server Code

In [ ]:
gtiff_bytes_str = read_tif_as_base64_str(tif_path)

In [ ]:
from rasterio.io import MemoryFile
from rasterio.plot import reshape_as_image

decoded_bytes = base64.b64decode(gtiff_bytes_str)
with MemoryFile(decoded_bytes) as memfile:
    with memfile.open() as src:
        meta = src.meta
        arr = reshape_as_image(src.read())



In [ ]:


if arr.dtype == "int16": # assuming it's read from araster and needs to be rescaled
    rescale_tif(arr)

In [ ]:
rescale_tif(arr)